In [4]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.1 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Copy your HTTPS Certificate to your local working directory (IMPORTANT)

In [8]:
!docker cp es01:/usr/share/elasticsearch/config/certs/http_ca.crt .


Test to see if your certificate is there

In [ ]:
!cat http_ca.crt

In [13]:
from elasticsearch import Elasticsearch
from tqdm.notebook import tqdm


In [32]:
# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "kC9v!!!!!!!!!!!!!!!!!jpQ2w" #Your Password from the initial steps of Elastic Search

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)


In [33]:
if es.ping():
    print("Connected to Elasticsearch cluster")
else:
    print("Could not connect to Elasticsearch")

Connected to Elasticsearch cluster


In [ ]:
# Test your connection

In [34]:
# Index name
index_name = "wiki-summary"

# Document to be indexed
document = {
    "title": "Document Title",
    "text": "This is the content of the document."
}

# Index the document
response = es.index(index=index_name, body=document)

In [35]:
response

ObjectApiResponse({'_index': 'wiki-summary', '_id': '715i-I0BkRXLy0xEKZh3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 11, '_primary_term': 1})

# Test with the first 10 documents

In [23]:
file_path = "/Users/work/Downloads/without-punctuation/without-punctuation.txt"  # Replace with the path to your file

# Open the file and iterate through each line
count=0
num_lines = sum(1 for line in open(file_path, "r"))


with open(file_path, "r") as file:
    for line in tqdm(file, total=num_lines, desc="Reading file", unit="lines"):
        count=count+1
        if count>10:
            break
        # Split the line based on the separator "|||"
        parts = line.strip().split("|||")
        
        # Assuming you want to access the first and second parts separately
        if len(parts) == 2:
            title = parts[0].strip()
            text = parts[1].strip()
            
            document = {
                "title": title,
                "text": text
            }
            response = es_client.index(index=index_name, body=document)
            
        else:
            # Handle lines that don't have exactly two parts separated by "|||"
            print("Invalid line format:", line)

Reading file:   0%|          | 0/5315384 [00:00<?, ?lines/s]

Perform a Query

In [42]:
print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    # Retrieve the score
    score = hit["_score"]
    # Retrieve other fields from the source
    title = hit["_source"]["title"]
    text = hit["_source"]["text"]
    print("Score: %.2f | Tile: %s | Text: %s" % (score,title, text))

Got 1 Hits:
Score: 5.42 | Tile: abraham lincoln | Text: abraham lincoln february 12 1809 april 15 1865 was an american politician and lawyer who served as the 16th president of the united states from march 1861 until his assassination in april 1865 lincoln led the united states through its civil war—its bloodiest war and perhaps its greatest moral constitutional and political crisis in doing so he preserved the union paved the way to the abolition of slavery strengthened the federal government and modernized the economy born in hodgenville kentucky lincoln grew up on the western frontier in kentucky and indiana largely self-educated he became a lawyer in illinois a whig party leader and was elected to the illinois house of representatives in which he served for eight years elected to the united states house of representatives in 1846 lincoln promoted rapid modernization of the economy and opposed the mexican–american war after a single term he returned to illinois and resumed his succe

In [26]:
response

ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [ ]:
file_path = "/Users/work/Downloads/without-punctuation/without-punctuation.txt"  # Replace with the path to your file

# Open the file and iterate through each line
num_lines = sum(1 for line in open(file_path, "r"))


with open(file_path, "r") as file:
    for line in tqdm(file, total=num_lines, desc="Reading file", unit="lines"):
        # Split the line based on the separator "|||"
        parts = line.strip().split("|||")
        
        # Assuming you want to access the first and second parts separately
        if len(parts) == 2:
            title = parts[0].strip()
            text = parts[1].strip()
            
            document = {
                "title": title,
                "text": text
            }
            response = es_client.index(index=index_name, body=document)
            
        else:
            # Handle lines that don't have exactly two parts separated by "|||"
            print("Invalid line format:", line)

Reading file:   0%|          | 0/5315384 [00:00<?, ?lines/s]

Invalid line format: unary numeral system ||| the unary numeral system is the bijective base-1 numeral system it is the simplest numeral system to represent natural numbers in order to represent a number n an arbitrarily chosen symbol representing 1 is repeated n times for examples the numbers 1 2 3 4 5 would be represented in this system as 1 11 111 1111 11111 these numbers should be distinguished from repunits which are also written as sequences of ones but have their usual decimal numerical interpretation this system is used in tallying for example using the tally mark | the number 3 is represented as ||| in east asian cultures the number three is represented as “ 三 ” a character that is drawn with three strokes



In [ ]:
## Write your wrapper function into DSPY to return as the proper format

In [ ]:
class YouRM(dspy.Retrieve):
    def __init__(self, es_client, es_index, es_password=None, k=3):
        super().__init__(k=k)
        if not es_password and not os.environ.get("ELASTIC_PASSWORD"):
            raise RuntimeError("You must supply ELASTIC_PASSWORD or set environment variable ELASTIC_PASSWORD")
        elif es_password:
            self.es_password = es_password
        else:
            self.es_password = os.environ["ELASTIC_PASSWORD"]
            
    def forward(self, query, k: Optional[int] = None) -> dspy.Prediction:
        """Search with You.com for top k passages for query or queries

        Args:
            query_or_queries (Union[str, List[str]]): The query or queries to search for.
            k (Optional[int]): The number of context strings to return, if not already specified in self.k

        Returns:
            dspy.Prediction: An object containing the retrieved passages.
        """


        k = k if k is not None else self.k

        passages = []
        
        # Define the index to search
        index_name = es_index #the name of the index of your elastic-search-dump

        # Define the search query
        search_query = {
            "query": {
                "match": {
                    "text": query  #took for granted that your index has : title, text as document format
                }
            }
        }

        # Perform the search
        response = es_client.search(index=index_name, body=search_query)

        for hit in response['hits']['hits']:
            
            #Uncomment for debug...
            # Retrieve the score
            #score = hit["_score"]
            # Retrieve other fields from the source
            #title = hit["_source"]["title"]
            text = hit["_source"]["text"]
            #print("Score: %.2f | Tile: %s | Text: %s" % (score,title, text))
            passages.append(text)
            if len(passages) == k:  # Break the loop once k documents are retrieved
                break

        return dspy.Prediction(passages=passages)